## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [7]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=2)

# Get the date one week ago
one_week_ago = today - timedelta(days=8)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

May. 17th May. 25th


In [8]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [9]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [10]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [11]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [12]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,May-21-2025,Condo/Co-op,1907 NE 4th St #8,Deerfield Beach,FL,33441.0,255000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/1907...,Beaches MLS,F10498131,N,Y,26.318456,-80.079643


In [13]:
print('Input county name:')
county = input()

Input county name:
Broward


In [14]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [11]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [15]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [16]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [17]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-05-20    32
2025-05-22    21
2025-05-21    19
2025-05-23    17
Name: count, dtype: int64

In [18]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [19]:
df_top_ten = df2.head(10)

In [20]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
65,PAST SALE,2025-05-20,Condo/Co-op,2200 S Ocean Ln #1406,Fort Lauderdale,FL,33316.0,1695000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10473379,N,Y,26.095913,-80.106280
90,PAST SALE,2025-05-21,Condo/Co-op,3100 NE 48th St #305,Fort Lauderdale,FL,33308.0,775000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10422060,N,Y,26.188089,-80.103925
71,PAST SALE,2025-05-20,Condo/Co-op,5200 N Ocean Blvd Unit 608B,Lauderdale By The Sea,FL,33308.0,725000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,MARMLS,A11710800,N,Y,26.201000,-80.094629
24,PAST SALE,2025-05-22,Condo/Co-op,1051 Hillsboro Mile Unit 910E,Hillsboro Beach,FL,33062.0,710000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1051...,Beaches MLS,F10495664,N,Y,26.279840,-80.079491
18,PAST SALE,2025-05-21,Condo/Co-op,2080 S Ocean Dr #1409,Hallandale Beach,FL,33009.0,680000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/208...,MARMLS,A11757793,N,Y,25.977222,-80.119313
16,PAST SALE,2025-05-21,Condo/Co-op,3500 Galt Ocean Dr #2402,Fort Lauderdale,FL,33308.0,670000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3500...,Beaches MLS,F10486175,N,Y,26.171499,-80.098121
46,PAST SALE,2025-05-20,Condo/Co-op,1938 Jackson St #504,Hollywood,FL,33020.0,565000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1938-Jacks...,Beaches MLS,F10483093,N,Y,26.008526,-80.146277
15,PAST SALE,2025-05-21,Condo/Co-op,615 N Riverside Dr #405,Pompano Beach,FL,33062.0,525000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/615-N-...,Beaches MLS,F10494976,N,Y,26.242183,-80.091074
43,PAST SALE,2025-05-20,Condo/Co-op,2030 S Ocean Dr #1906,Hallandale Beach,FL,33009.0,520000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/203...,MARMLS,A11754919,N,Y,25.978359,-80.119546
21,PAST SALE,2025-05-23,Condo/Co-op,5100 Dupont Blvd Unit 8D,Fort Lauderdale,FL,33308.0,500000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/5100...,Beaches MLS,F10466384,N,Y,26.190912,-80.103378


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [21]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [22]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [23]:
response_df = pd.DataFrame(response_list)

In [24]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [25]:
df_top_ten = merged_df

In [26]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-05-20,Condo/Co-op,2200 S Ocean Ln #1406,Fort Lauderdale,FL,33316.0,1695000.0,2.0,2.5,...,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Wm Mobley McClellan,Fisher McClellan Real Estate,None,None,Martha Buckley,Coldwell Banker Realty,None,None
1,PAST SALE,2025-05-21,Condo/Co-op,3100 NE 48th St #305,Fort Lauderdale,FL,33308.0,775000.0,3.0,3.0,...,-80.103925,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Niliana Nassar-Garcia,Engel & Voelkers Fort Lauderdale,Francisco Garcia,Engel & Voelkers Fort Lauderdale,Lourdes Maestres,"Compass Florida, LLC",None,"Compass Florida, LLC"
2,PAST SALE,2025-05-20,Condo/Co-op,5200 N Ocean Blvd Unit 608B,Lauderdale By The Sea,FL,33308.0,725000.0,2.0,2.0,...,-80.094629,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Alix Aranza,One Sotheby's Int'l Realty,None,None,None,None,None,None
3,PAST SALE,2025-05-22,Condo/Co-op,1051 Hillsboro Mile Unit 910E,Hillsboro Beach,FL,33062.0,710000.0,2.0,2.0,...,-80.079491,https://www.redfin.com/FL/Hillsboro-Beach/1051...,Denise Stewart,Keller Williams Realty Profess,Dennis Stewart,Keller Williams Realty Profess,Sheri Wells,Re/Max 1st Choice,None,None
4,PAST SALE,2025-05-21,Condo/Co-op,2080 S Ocean Dr #1409,Hallandale Beach,FL,33009.0,680000.0,2.0,2.0,...,-80.119313,https://www.redfin.com/FL/Hallandale-Beach/208...,Bill Ioannou,"Adelphia Realty, LLC.",None,None,Victor Persia,Compra Venta Miami,None,None
5,PAST SALE,2025-05-21,Condo/Co-op,3500 Galt Ocean Dr #2402,Fort Lauderdale,FL,33308.0,670000.0,2.0,2.0,...,-80.098121,https://www.redfin.com/FL/Fort-Lauderdale/3500...,Katherine Pantelis,Atlantic Properties Int'l Inc,None,None,Carlos Nolasco,Galt Ocean Realty Inc,None,None
6,PAST SALE,2025-05-20,Condo/Co-op,1938 Jackson St #504,Hollywood,FL,33020.0,565000.0,2.0,2.0,...,-80.146277,https://www.redfin.com/FL/Hollywood/1938-Jacks...,Angelina Tedesco,"Compass Florida, LLC",None,None,Annie Lavallee,RE/MAX 5 Star Realty,None,None
7,PAST SALE,2025-05-21,Condo/Co-op,615 N Riverside Dr #405,Pompano Beach,FL,33062.0,525000.0,2.0,2.0,...,-80.091074,https://www.redfin.com/FL/Pompano-Beach/615-N-...,Linda Walton,RE/MAX Experience,None,None,Adela Philipson,Philipson Realty LLC,None,None
8,PAST SALE,2025-05-20,Condo/Co-op,2030 S Ocean Dr #1906,Hallandale Beach,FL,33009.0,520000.0,2.0,2.0,...,-80.119546,https://www.redfin.com/FL/Hallandale-Beach/203...,Brandon Talalaevsky,"Property Pro Partners, LLC.",None,None,Bojana Lambrou,Coldwell Banker Realty,None,None
9,PAST SALE,2025-05-23,Condo/Co-op,5100 Dupont Blvd Unit 8D,Fort Lauderdale,FL,33308.0,500000.0,2.0,2.5,...,-80.103378,https://www.redfin.com/FL/Fort-Lauderdale/5100...,Michael Palmer,The Keyes Company,None,None,Lanorris Miller,"Elite Realty Partners, Inc.",None,None


## Current Week's Values

In [27]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

89


In [28]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$287,503


In [29]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$250


In [30]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$25,587,800


In [31]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           89              287503          250                     25587800.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [32]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
115
-----------
Input Previous Week Condo Average Sales Price:
325860
-----------
Input Previous Week Condo Average PSF:
273
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
37473900.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [33]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [34]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [35]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [36]:
pd.set_option('display.max_columns',None)

In [37]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-20,Condo/Co-op,2200 S Ocean Ln #1406,Fort Lauderdale,FL,33316.0,1695000.0,2.0,2.5,Point Of Americas Ii,2071.0,NaN,1971.0,NaN,818.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10473379,N,Y,26.095913,-80.10628,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Wm Mobley McClellan,Fisher McClellan Real Estate,None,None,Martha Buckley,Coldwell Banker Realty,None,None,1,orange


In [38]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [39]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-20,Condo/Co-op,2200 S Ocean Ln #1406,Fort Lauderdale,FL,33316.0,1695000.0,2.0,2.5,Point Of Americas Ii,2071.0,NaN,1971.0,NaN,818.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10473379,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Wm Mobley McClellan,Fisher McClellan Real Estate,None,None,Martha Buckley,Coldwell Banker Realty,None,None,1,orange
1,PAST SALE,2025-05-21,Condo/Co-op,3100 NE 48th St #305,Fort Lauderdale,FL,33308.0,775000.0,3.0,3.0,Pilot House Condo,2268.0,NaN,1977.0,NaN,342.0,1673.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10422060,N,Y,26.188089,-80.103925,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Niliana Nassar-Garcia,Engel & Voelkers Fort Lauderdale,Francisco Garcia,Engel & Voelkers Fort Lauderdale,Lourdes Maestres,"Compass Florida, LLC",None,"Compass Florida, LLC",2,blue
2,PAST SALE,2025-05-20,Condo/Co-op,5200 N Ocean Blvd Unit 608B,Lauderdale By The Sea,FL,33308.0,725000.0,2.0,2.0,SEA RANCH LAKES NORTH CON,1330.0,NaN,1971.0,NaN,545.0,1089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,MARMLS,A11710800,N,Y,26.201000,-80.094629,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Alix Aranza,One Sotheby's Int'l Realty,None,None,None,None,None,None,3,blue
3,PAST SALE,2025-05-22,Condo/Co-op,1051 Hillsboro Mile Unit 910E,Hillsboro Beach,FL,33062.0,710000.0,2.0,2.0,Hillsboro Lebaron,1200.0,NaN,1969.0,NaN,592.0,1238.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1051...,Beaches MLS,F10495664,N,Y,26.279840,-80.079491,https://www.redfin.com/FL/Hillsboro-Beach/1051...,Denise Stewart,Keller Williams Realty Profess,Dennis Stewart,Keller Williams Realty Profess,Sheri Wells,Re/Max 1st Choice,None,None,4,blue
4,PAST SALE,2025-05-21,Condo/Co-op,2080 S Ocean Dr #1409,Hallandale Beach,FL,33009.0,680000.0,2.0,2.0,2080 Ocean Drive,NaN,NaN,2003.0,NaN,NaN,1264.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/208...,MARMLS,A11757793,N,Y,25.977222,-80.119313,https://www.redfin.com/FL/Hallandale-Beach/208...,Bill Ioannou,"Adelphia Realty, LLC.",None,None,Victor Persia,Compra Venta Miami,None,None,5,blue
5,PAST SALE,2025-05-21,Condo/Co-op,3500 Galt Ocean Dr #2402,Fort Lauderdale,FL,33308.0,670000.0,2.0,2.0,Playa Del Sol Condo,1435.0,NaN,1974.0,NaN,467.0,1402.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3500...,Beaches MLS,F10486175,N,Y,26.171499,-80.098121,https://www.redfin.com/FL/Fort-Lauderdale/3500...,Katherine Pantelis,Atlantic Properties Int'l Inc,None,None,Carlos Nolasco,Galt Ocean Realty Inc,None,None,6,blue
6,PAST SALE,2025-05-20,Condo/Co-op,1938 Jackson St #504,Hollywood,FL,33020.0,565000.0,2.0,2.0,Aesthetic Apartments,1710.0,NaN,2023.0,NaN,330.0,684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1938-Jacks...,Beaches MLS,F10483093,N,Y,26.008526,-80.146277,https://www.redfin.com/FL/Hollywood/1938-Jacks...,Angelina Tedesco,"Compass Florida, LLC",None,None,Annie Lavallee,RE/MAX 5 Star Realty,None,None,7,blue
7,PAST SALE,2025-05-21,Condo/Co-op,615 N Riverside Dr #405,Pompano Beach,FL,33062.0,525000.0,2.0,2.0,Riverside Terrace,1600.0,NaN,1969.0,NaN,328.0,592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/615-N-...,Beaches MLS,F10494976,N,Y,26.242183,-80.091074,https://www.redfin.com/FL/Pompano-Beach/615-N-...,Linda Walton,RE/MAX Experience,None,None,Adela Philipson,Philipson Realty LLC,None,None,8,blue
8,PAST SALE,2025-05-20,Condo/Co-op,2030 S Ocean Dr #1906,Hallandale Beach,FL,33009.0,520000.0,2.0,2.0,

In [40]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [41]:
m.save('index.html')

## Data snagger

In [42]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [43]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-20,Condo/Co-op,2200 S Ocean Ln #1406,Fort Lauderdale,FL,33316.0,1695000.0,2.0,2.5,Point Of Americas Ii,2071.0,NaN,1971.0,NaN,818.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10473379,N,Y,26.095913,-80.10628,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Wm Mobley McClellan,Fisher McClellan Real Estate,None,None,Martha Buckley,Coldwell Banker Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [44]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_05272025


# CREATE TEMPLATE 

In [45]:
muni_set = set(df_top_ten['CITY'])

In [46]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [47]:
df_top_ten.reset_index(inplace=True,drop=True)

In [48]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [49]:
top_sale

'Point Of Americas Ii at 2200 S Ocean Ln #1406 in Fort Lauderdale'

In [50]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [51]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [52]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-20,Condo/Co-op,2200 S Ocean Ln #1406,Fort Lauderdale,FL,33316.0,1695000.0,2.0,2.5,Point Of Americas Ii,2071.0,NaN,1971.0,NaN,818.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10473379,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Wm Mobley McClellan,Fisher McClellan Real Estate,None,None,Martha Buckley,Coldwell Banker Realty,None,None,1,orange,Point Of Americas Ii at 2200 S Ocean Ln #1406 ...
1,PAST SALE,2025-05-21,Condo/Co-op,3100 NE 48th St #305,Fort Lauderdale,FL,33308.0,775000.0,3.0,3.0,Pilot House Condo,2268.0,NaN,1977.0,NaN,342.0,1673.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10422060,N,Y,26.188089,-80.103925,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Niliana Nassar-Garcia,Engel & Voelkers Fort Lauderdale,Francisco Garcia,Engel & Voelkers Fort Lauderdale,Lourdes Maestres,"Compass Florida, LLC",None,"Compass Florida, LLC",2,blue,Pilot House Condo at 3100 NE 48th St #305 in F...
2,PAST SALE,2025-05-20,Condo/Co-op,5200 N Ocean Blvd Unit 608B,Lauderdale By The Sea,FL,33308.0,725000.0,2.0,2.0,SEA RANCH LAKES NORTH CON,1330.0,NaN,1971.0,NaN,545.0,1089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,MARMLS,A11710800,N,Y,26.201000,-80.094629,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Alix Aranza,One Sotheby's Int'l Realty,None,None,None,None,None,None,3,blue,SEA RANCH LAKES NORTH CON at 5200 N Ocean Blvd...
3,PAST SALE,2025-05-22,Condo/Co-op,1051 Hillsboro Mile Unit 910E,Hillsboro Beach,FL,33062.0,710000.0,2.0,2.0,Hillsboro Lebaron,1200.0,NaN,1969.0,NaN,592.0,1238.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1051...,Beaches MLS,F10495664,N,Y,26.279840,-80.079491,https://www.redfin.com/FL/Hillsboro-Beach/1051...,Denise Stewart,Keller Williams Realty Profess,Dennis Stewart,Keller Williams Realty Profess,Sheri Wells,Re/Max 1st Choice,None,None,4,blue,Hillsboro Lebaron at 1051 Hillsboro Mile Unit ...
4,PAST SALE,2025-05-21,Condo/Co-op,2080 S Ocean Dr #1409,Hallandale Beach,FL,33009.0,680000.0,2.0,2.0,2080 Ocean Drive,NaN,NaN,2003.0,NaN,NaN,1264.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/208...,MARMLS,A11757793,N,Y,25.977222,-80.119313,https://www.redfin.com/FL/Hallandale-Beach/208...,Bill Ioannou,"Adelphia Realty, LLC.",None,None,Victor Persia,Compra Venta Miami,None,None,5,blue,2080 Ocean Drive at 2080 S Ocean Dr #1409 in H...
5,PAST SALE,2025-05-21,Condo/Co-op,3500 Galt Ocean Dr #2402,Fort Lauderdale,FL,33308.0,670000.0,2.0,2.0,Playa Del Sol Condo,1435.0,NaN,1974.0,NaN,467.0,1402.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3500...,Beaches MLS,F10486175,N,Y,26.171499,-80.098121,https://www.redfin.com/FL/Fort-Lauderdale/3500...,Katherine Pantelis,Atlantic Properties Int'l Inc,None,None,Carlos Nolasco,Galt Ocean Realty Inc,None,None,6,blue,Playa Del Sol Condo at 3500 Galt Ocean Dr #240...
6,PAST SALE,2025-05-20,Condo/Co-op,1938 Jackson St #504,Hollywood,FL,33020.0,565000.0,2.0,2.0,Aesthetic Apartments,1710.0,NaN,2023.0,NaN,330.0,684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1938-Jacks...,Beaches MLS,F10483093,N,Y,26.008526,-80.146277,https://www.redfin.com/FL/Hollywood/1938-Jacks...,Angelina Tedesco,"Compass Florida, LLC",None,None,Annie Lavallee,RE/MAX 5 Star Realty,None,None,7,blue,Aesthetic Apartments at 1938 Jackson St #504 i...
7,PAST SALE,2025-05-21,Condo/Co-op,615 N Riverside Dr #405,Pompano Beach,FL,33062.0,525000.0,2.0,2.0,Riverside Terrace,1600.0,NaN,1969.0,NaN,328.0,592

In [53]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Point Of Americas Ii closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $500,000 to $1,695,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Pompano Beach, Fort Lauderdale, Hollywood, Lauderdale By The Sea, Hillsboro Beach, Hallandale Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 89 condo sales totaling $25,587,800 million from May. 17th to May. 25th. The previous week, brokers closed 115 condo sales totaling $37,473,900.

Last week’s units sold for an average of $287,503, lower than the $325

In [54]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [55]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [56]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [57]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-20,Condo/Co-op,2200 S Ocean Ln #1406,Fort Lauderdale,FL,33316.0,1695000.0,2.0,2.5,Point Of Americas Ii,2071.0,NaN,1971.0,NaN,818.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10473379,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Wm Mobley McClellan,Fisher McClellan Real Estate,None,None,Martha Buckley,Coldwell Banker Realty,None,None,1,orange,Point Of Americas Ii at 2200 S Ocean Ln #1406 ...
1,PAST SALE,2025-05-21,Condo/Co-op,3100 NE 48th St #305,Fort Lauderdale,FL,33308.0,775000.0,3.0,3.0,Pilot House Condo,2268.0,NaN,1977.0,NaN,342.0,1673.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10422060,N,Y,26.188089,-80.103925,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Niliana Nassar-Garcia,Engel & Voelkers Fort Lauderdale,Francisco Garcia,Engel & Voelkers Fort Lauderdale,Lourdes Maestres,"Compass Florida, LLC",None,"Compass Florida, LLC",2,blue,Pilot House Condo at 3100 NE 48th St #305 in F...
2,PAST SALE,2025-05-20,Condo/Co-op,5200 N Ocean Blvd Unit 608B,Lauderdale By The Sea,FL,33308.0,725000.0,2.0,2.0,SEA RANCH LAKES NORTH CON,1330.0,NaN,1971.0,NaN,545.0,1089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,MARMLS,A11710800,N,Y,26.201000,-80.094629,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Alix Aranza,One Sotheby's Int'l Realty,None,None,None,None,None,None,3,blue,SEA RANCH LAKES NORTH CON at 5200 N Ocean Blvd...
3,PAST SALE,2025-05-22,Condo/Co-op,1051 Hillsboro Mile Unit 910E,Hillsboro Beach,FL,33062.0,710000.0,2.0,2.0,Hillsboro Lebaron,1200.0,NaN,1969.0,NaN,592.0,1238.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1051...,Beaches MLS,F10495664,N,Y,26.279840,-80.079491,https://www.redfin.com/FL/Hillsboro-Beach/1051...,Denise Stewart,Keller Williams Realty Profess,Dennis Stewart,Keller Williams Realty Profess,Sheri Wells,Re/Max 1st Choice,None,None,4,blue,Hillsboro Lebaron at 1051 Hillsboro Mile Unit ...
4,PAST SALE,2025-05-21,Condo/Co-op,2080 S Ocean Dr #1409,Hallandale Beach,FL,33009.0,680000.0,2.0,2.0,2080 Ocean Drive,NaN,NaN,2003.0,NaN,NaN,1264.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/208...,MARMLS,A11757793,N,Y,25.977222,-80.119313,https://www.redfin.com/FL/Hallandale-Beach/208...,Bill Ioannou,"Adelphia Realty, LLC.",None,None,Victor Persia,Compra Venta Miami,None,None,5,blue,2080 Ocean Drive at 2080 S Ocean Dr #1409 in H...
5,PAST SALE,2025-05-21,Condo/Co-op,3500 Galt Ocean Dr #2402,Fort Lauderdale,FL,33308.0,670000.0,2.0,2.0,Playa Del Sol Condo,1435.0,NaN,1974.0,NaN,467.0,1402.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3500...,Beaches MLS,F10486175,N,Y,26.171499,-80.098121,https://www.redfin.com/FL/Fort-Lauderdale/3500...,Katherine Pantelis,Atlantic Properties Int'l Inc,None,None,Carlos Nolasco,Galt Ocean Realty Inc,None,None,6,blue,Playa Del Sol Condo at 3500 Galt Ocean Dr #240...
6,PAST SALE,2025-05-20,Condo/Co-op,1938 Jackson St #504,Hollywood,FL,33020.0,565000.0,2.0,2.0,Aesthetic Apartments,1710.0,NaN,2023.0,NaN,330.0,684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1938-Jacks...,Beaches MLS,F10483093,N,Y,26.008526,-80.146277,https://www.redfin.com/FL/Hollywood/1938-Jacks...,Angelina Tedesco,"Compass Florida, LLC",None,None,Annie Lavallee,RE/MAX 5 Star Realty,None,None,7,blue,Aesthetic Apartments at 1938 Jackson St #504 i...
7,PAST SALE,2025-05-21,Condo/Co-op,615 N Riverside Dr #405,Pompano Beach,FL,33062.0,525000.0,2.0,2.0,Riverside Terrace,1600.0,NaN,1969.0,NaN,328.0,592

In [58]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Point Of Americas Ii closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $500,000 to $1,695,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Pompano Beach, Fort Lauderdale, Hollywood, Lauderdale By The Sea, Hillsboro Beach, Hallandale Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 89 condo sales totaling $25,587,800 million from May. 17th to May. 25th. The previous week, brokers closed 115 condo sales totaling $37,473,900.

Last week’s units sold for an average of $287,503, lower than the $325

In [66]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Fort-Lauderdale/2200-S-Ocean-Ln-33316/unit-1406/home/41805813


In [73]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/3100-NE-48th-St-33308/unit-305/home/41704430


In [74]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/5100-DuPont-Blvd-33308/unit-8D/home/41706097


In [75]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Hallandale-Beach/2030-S-Ocean-Dr-33009/unit-1906/home/41966812


In [76]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/2200-S-Ocean-Ln-33316/unit-1406/home/41805813


In [77]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Pompano-Beach/615-N-Riverside-Dr-33062/unit-405/home/41528315


In [78]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-20,Condo/Co-op,2200 S Ocean Ln #1406,Fort Lauderdale,FL,33316.0,1695000.0,2.0,2.5,Point Of Americas Ii,2071.0,NaN,1971.0,NaN,818.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10473379,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Wm Mobley McClellan,Fisher McClellan Real Estate,None,None,Martha Buckley,Coldwell Banker Realty,None,None,1,orange,Point Of Americas Ii at 2200 S Ocean Ln #1406 ...
1,PAST SALE,2025-05-21,Condo/Co-op,3100 NE 48th St #305,Fort Lauderdale,FL,33308.0,775000.0,3.0,3.0,Pilot House Condo,2268.0,NaN,1977.0,NaN,342.0,1673.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10422060,N,Y,26.188089,-80.103925,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Niliana Nassar-Garcia,Engel & Voelkers Fort Lauderdale,Francisco Garcia,Engel & Voelkers Fort Lauderdale,Lourdes Maestres,"Compass Florida, LLC",None,"Compass Florida, LLC",2,blue,Pilot House Condo at 3100 NE 48th St #305 in F...
2,PAST SALE,2025-05-20,Condo/Co-op,5200 N Ocean Blvd Unit 608B,Lauderdale By The Sea,FL,33308.0,725000.0,2.0,2.0,SEA RANCH LAKES NORTH CON,1330.0,NaN,1971.0,NaN,545.0,1089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,MARMLS,A11710800,N,Y,26.201000,-80.094629,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Alix Aranza,One Sotheby's Int'l Realty,None,None,None,None,None,None,3,blue,SEA RANCH LAKES NORTH CON at 5200 N Ocean Blvd...
3,PAST SALE,2025-05-22,Condo/Co-op,1051 Hillsboro Mile Unit 910E,Hillsboro Beach,FL,33062.0,710000.0,2.0,2.0,Hillsboro Lebaron,1200.0,NaN,1969.0,NaN,592.0,1238.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1051...,Beaches MLS,F10495664,N,Y,26.279840,-80.079491,https://www.redfin.com/FL/Hillsboro-Beach/1051...,Denise Stewart,Keller Williams Realty Profess,Dennis Stewart,Keller Williams Realty Profess,Sheri Wells,Re/Max 1st Choice,None,None,4,blue,Hillsboro Lebaron at 1051 Hillsboro Mile Unit ...
4,PAST SALE,2025-05-21,Condo/Co-op,2080 S Ocean Dr #1409,Hallandale Beach,FL,33009.0,680000.0,2.0,2.0,2080 Ocean Drive,NaN,NaN,2003.0,NaN,NaN,1264.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/208...,MARMLS,A11757793,N,Y,25.977222,-80.119313,https://www.redfin.com/FL/Hallandale-Beach/208...,Bill Ioannou,"Adelphia Realty, LLC.",None,None,Victor Persia,Compra Venta Miami,None,None,5,blue,2080 Ocean Drive at 2080 S Ocean Dr #1409 in H...
5,PAST SALE,2025-05-21,Condo/Co-op,3500 Galt Ocean Dr #2402,Fort Lauderdale,FL,33308.0,670000.0,2.0,2.0,Playa Del Sol Condo,1435.0,NaN,1974.0,NaN,467.0,1402.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3500...,Beaches MLS,F10486175,N,Y,26.171499,-80.098121,https://www.redfin.com/FL/Fort-Lauderdale/3500...,Katherine Pantelis,Atlantic Properties Int'l Inc,None,None,Carlos Nolasco,Galt Ocean Realty Inc,None,None,6,blue,Playa Del Sol Condo at 3500 Galt Ocean Dr #240...
6,PAST SALE,2025-05-20,Condo/Co-op,1938 Jackson St #504,Hollywood,FL,33020.0,565000.0,2.0,2.0,Aesthetic Apartments,1710.0,NaN,2023.0,NaN,330.0,684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1938-Jacks...,Beaches MLS,F10483093,N,Y,26.008526,-80.146277,https://www.redfin.com/FL/Hollywood/1938-Jacks...,Angelina Tedesco,"Compass Florida, LLC",None,None,Annie Lavallee,RE/MAX 5 Star Realty,None,None,7,blue,Aesthetic Apartments at 1938 Jackson St #504 i...
7,PAST SALE,2025-05-21,Condo/Co-op,615 N Riverside Dr #405,Pompano Beach,FL,33062.0,525000.0,2.0,2.0,Riverside Terrace,1600.0,NaN,1969.0,NaN,328.0,592

## Time on Market Calculator

In [80]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 2, 7) ## List (Earlier) date
date2 = datetime(2025, 5, 21) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

469


## Clean CSV for Datawrapper Chart

In [81]:
chart_df = df_top_ten

In [82]:
chart_df = chart_df.fillna(" ")

In [83]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [84]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [85]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [86]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [94]:
# Coerce non-numeric to NaN, then you’ll have a float column
chart_df['$/SQUARE FEET'] = pd.to_numeric(chart_df['$/SQUARE FEET'], errors='coerce')

In [98]:
chart_df['SQUARE FEET'] = pd.to_numeric(chart_df['SQUARE FEET'], errors='coerce')

In [96]:
chart_df['$/SQUARE FEET'].astype(float)

0    818.0
1    342.0
2    545.0
3    592.0
4      NaN
5    467.0
6    330.0
7    328.0
8    396.0
9    396.0
Name: $/SQUARE FEET, dtype: float64

In [101]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [102]:
csv_date_string = today.strftime("%m_%d_%Y")

In [103]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [104]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-05-20,2200 S Ocean Ln #1406 in Fort Lauderdale,"$1,695,000",2,2.5,"2,071",1971,$818,Beaches MLS,F10473379,Wm Mobley McClellan Fisher McClellan Real Esta...,Martha Buckley Coldwell Banker Realty
1,2025-05-21,3100 NE 48th St #305 in Fort Lauderdale,"$775,000",3,3.0,"2,268",1977,$342,Beaches MLS,F10422060,Niliana Nassar-Garcia Engel & Voelkers Fort La...,"Lourdes Maestres Compass Florida, LLC Compas..."
2,2025-05-20,5200 N Ocean Blvd Unit 608B in Lauderdale By T...,"$725,000",2,2.0,"1,330",1971,$545,MARMLS,A11710800,Alix Aranza One Sotheby's Int'l Realty,
3,2025-05-22,1051 Hillsboro Mile Unit 910E in Hillsboro Beach,"$710,000",2,2.0,"1,200",1969,$592,Beaches MLS,F10495664,Denise Stewart Keller Williams Realty Profess ...,Sheri Wells Re/Max 1st Choice
4,2025-05-21,2080 S Ocean Dr #1409 in Hallandale Beach,"$680,000",2,2.0,nan,2003,NaN,MARMLS,A11757793,"Bill Ioannou Adelphia Realty, LLC.",Victor Persia Compra Venta Miami
5,2025-05-21,3500 Galt Ocean Dr #2402 in Fort Lauderdale,"$670,000",2,2.0,"1,435",1974,$467,Beaches MLS,F10486175,Katherine Pantelis Atlantic Properties Int'l I...,Carlos Nolasco Galt Ocean Realty Inc
6,2025-05-20,1938 Jackson St #504 in Hollywood,"$565,000",2,2.0,"1,710",2023,$330,Beaches MLS,F10483093,"Angelina Tedesco Compass Florida, LLC",Annie Lavallee RE/MAX 5 Star Realty
7,2025-05-21,615 N Riverside Dr #405 in Pompano Beach,"$525,000",2,2.0,"1,600",1969,$328,Beaches MLS,F10494976,Linda Walton RE/MAX Experience,Adela Philipson Philipson Realty LLC
8,2025-05-20,2030 S Ocean Dr #1906 in Hallandale Beach,"$520,000",2,2.0,"1,312",1970,$396,MARMLS,A11754919,"Brandon Talalaevsky Property Pro Partners, LLC...",Bojana Lambrou Coldwell Banker Realty
9,2025-05-23,5100 Dupont Blvd Unit 8D in Fort Lauderdale,"$500,000",2,2.5,"1,262",1966,$396,Beaches MLS,F10466384,Michael Palmer The Keyes Company,"Lanorris Miller Elite Realty Partners, Inc."
